In [71]:
# Import dependencies
import pandas as pd
import numpy as np
import seaborn as sns
pd.set_option('max_colwidth', 400)

In [72]:
# Read the data into a Pandas DataFrame
chronic_disease_df = pd.read_csv('Resources/U.S._Chronic_Disease_Indicators 2.csv')
chronic_disease_df.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,TopicID,QuestionID,ResponseID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
0,2019,2019,AR,Arkansas,BRFSS,Diabetes,Diabetes among adults,NaN,%,Crude Prevalence,...,DIA,DIA01,NaN,CRDPREV,SEX,SEXM,NaN,NaN,NaN,NaN
1,2019,2019,ID,Idaho,BRFSS,Diabetes,Diabetes among adults,NaN,%,Crude Prevalence,...,DIA,DIA01,NaN,CRDPREV,SEX,SEXM,NaN,NaN,NaN,NaN
2,2019,2019,IN,Indiana,YRBSS,Sleep,Short sleep duration among high school students,NaN,%,Crude Prevalence,...,SLEP,SLP02,NaN,CRDPREV,GRADE,GRD 12.00,NaN,NaN,NaN,NaN
3,2019,2019,IA,Iowa,NVSS,Asthma,"Asthma mortality among all people, underlying cause",NaN,Number,Number,...,AST,AST01,NaN,NMBR,OVERALL,OVR,NaN,NaN,NaN,NaN
4,2019,2019,IA,Iowa,BRFSS,Asthma,Current asthma among adults,NaN,%,Crude Prevalence,...,AST,AST02,NaN,CRDPREV,AGE,AGE1844,NaN,NaN,NaN,NaN


In [73]:
chronic_disease_df.info

<bound method DataFrame.info of         YearStart  YearEnd LocationAbbr    LocationDesc DataSource  \
0            2019     2019           AR        Arkansas      BRFSS   
1            2019     2019           ID           Idaho      BRFSS   
2            2019     2019           IN         Indiana      YRBSS   
3            2019     2019           IA            Iowa       NVSS   
4            2019     2019           IA            Iowa      BRFSS   
...           ...      ...          ...             ...        ...   
309210       2022     2022           VI  Virgin Islands      BRFSS   
309211       2022     2022           WV   West Virginia      BRFSS   
309212       2022     2022           WI       Wisconsin      BRFSS   
309213       2022     2022           VT         Vermont      BRFSS   
309214       2022     2022           WA      Washington      BRFSS   

                                        Topic  \
0                                    Diabetes   
1                            

In [74]:
chronic_disease_df.columns

Index(['YearStart', 'YearEnd', 'LocationAbbr', 'LocationDesc', 'DataSource',
       'Topic', 'Question', 'Response', 'DataValueUnit', 'DataValueType',
       'DataValue', 'DataValueAlt', 'DataValueFootnoteSymbol',
       'DataValueFootnote', 'LowConfidenceLimit', 'HighConfidenceLimit',
       'StratificationCategory1', 'Stratification1', 'StratificationCategory2',
       'Stratification2', 'StratificationCategory3', 'Stratification3',
       'Geolocation', 'LocationID', 'TopicID', 'QuestionID', 'ResponseID',
       'DataValueTypeID', 'StratificationCategoryID1', 'StratificationID1',
       'StratificationCategoryID2', 'StratificationID2',
       'StratificationCategoryID3', 'StratificationID3'],
      dtype='object')

In [75]:
chronic_disease_df.rename(columns={
    'YearStart': 'year_started',
    'YearEnd': 'year_ended',
    'LocationAbbr': 'state_abbr',
    'LocationDesc': 'state'}, inplace=True)

chronic_disease_df

,year_started,year_ended,state_abbr,state,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,TopicID,QuestionID,ResponseID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
0,2019,2019,AR,Arkansas,BRFSS,Diabetes,Diabetes among adults,NaN,%,Crude Prevalence,...,DIA,DIA01,NaN,CRDPREV,SEX,SEXM,NaN,NaN,NaN,NaN
1,2019,2019,ID,Idaho,BRFSS,Diabetes,Diabetes among adults,NaN,%,Crude Prevalence,...,DIA,DIA01,NaN,CRDPREV,SEX,SEXM,NaN,NaN,NaN,NaN
2,2019,2019,IN,Indiana,YRBSS,Sleep,Short sleep duration among high school students,NaN,%,Crude Prevalence,...,SLEP,SLP02,NaN,CRDPREV,GRADE,GRD 12.00,NaN,NaN,NaN,NaN
3,2019,2019,IA,Iowa,NVSS,Asthma,"Asthma mortality among all people, underlying cause",NaN,Number,Number,...,AST,AST01,NaN,NMBR,OVERALL,OVR,NaN,NaN,NaN,NaN
4,2019,2019,IA,Iowa,BRFSS,Asthma,Current asthma among adults,NaN,%,Crude Prevalence,...,AST,AST02,NaN,CRDPREV,AGE,AGE1844,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309210,2022,2022,VI,Virgin Islands,BRFSS,Tobacco,Quit attempts in the past year among adult current smokers,NaN,%,Age-adjusted Prevalence,...,TOB,TOB06,NaN,AGEADJPREV,RACE,AIAN,NaN,NaN,NaN,NaN
309211,2022,2022,WV,West Virginia,BRFSS,Chronic Obstructive Pulmonary Disease,Chronic obstructive pulmonary disease among adults,NaN,%,Crude Prevalence,...,COPD,COPD01,NaN,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
309212,2022,2022,WI,Wisconsin,BRFSS,Immunization,Pneumococcal vaccination among adults aged 65 years and older,NaN,%,Crude Prevalence,...,IMM,IMM04,NaN,CRDPREV,RACE,BLK,NaN,NaN,NaN,NaN
309213,2022,2022,VT,Vermont,BRFSS,Social Determinants of Health,Lack of health insurance among adults aged 18-64 years,NaN,%,Crude Prevalence,...,SDOH,SDH09,NaN,CRDPREV,RACE,HAPI,NaN,NaN,NaN,NaN


In [76]:
cleaned_diseases_df = pd.DataFrame({
    
})

In [77]:
datasource_groupby = chronic_disease_df.groupby('DataSource')['Topic'].max()
datasource_groupby.head()

DataSource
ACS                               Social Determinants of Health
AEDS                                                    Alcohol
ANRF                                                    Tobacco
BRFSS                                                   Tobacco
CMS Part A Claims Data    Chronic Obstructive Pulmonary Disease
Name: Topic, dtype: object

In [78]:
datasource_groupby = chronic_disease_df[chronic_disease_df['DataSource'] == 'BRFSS']
datasource_groupby.head()

,year_started,year_ended,state_abbr,state,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,TopicID,QuestionID,ResponseID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
0,2019,2019,AR,Arkansas,BRFSS,Diabetes,Diabetes among adults,NaN,%,Crude Prevalence,...,DIA,DIA01,NaN,CRDPREV,SEX,SEXM,NaN,NaN,NaN,NaN
1,2019,2019,ID,Idaho,BRFSS,Diabetes,Diabetes among adults,NaN,%,Crude Prevalence,...,DIA,DIA01,NaN,CRDPREV,SEX,SEXM,NaN,NaN,NaN,NaN
4,2019,2019,IA,Iowa,BRFSS,Asthma,Current asthma among adults,NaN,%,Crude Prevalence,...,AST,AST02,NaN,CRDPREV,AGE,AGE1844,NaN,NaN,NaN,NaN
6,2019,2019,IA,Iowa,BRFSS,Health Status,Recent activity limitation among adults,NaN,Number,Crude Mean,...,HEA,HEA04,NaN,CRDMEAN,SEX,SEXF,NaN,NaN,NaN,NaN
7,2019,2019,IA,Iowa,BRFSS,Mental Health,Depression among adults,NaN,%,Crude Prevalence,...,MEN,MEN02,NaN,CRDPREV,RACE,MRC,NaN,NaN,NaN,NaN


In [79]:
chronic_disease_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309215 entries, 0 to 309214
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   year_started               309215 non-null  int64  
 1   year_ended                 309215 non-null  int64  
 2   state_abbr                 309215 non-null  object 
 3   state                      309215 non-null  object 
 4   DataSource                 309215 non-null  object 
 5   Topic                      309215 non-null  object 
 6   Question                   309215 non-null  object 
 7   Response                   0 non-null       float64
 8   DataValueUnit              309215 non-null  object 
 9   DataValueType              309215 non-null  object 
 10  DataValue                  209196 non-null  float64
 11  DataValueAlt               209196 non-null  float64
 12  DataValueFootnoteSymbol    101716 non-null  object 
 13  DataValueFootnote          10

In [80]:
chronic_disease_df_drop = chronic_disease_df.drop(columns=['Response','StratificationCategory2', 'Stratification2', 'StratificationCategory3', 'Stratification3',
                                 'ResponseID', 'StratificationCategoryID2', 'StratificationID2', 'StratificationCategoryID3', 'StratificationID3',
                                 'DataValueFootnoteSymbol', 'DataValueFootnote', 'LowConfidenceLimit', 'HighConfidenceLimit', 'TopicID', 'QuestionID', 
                                 'DataValueTypeID'])
chronic_disease_df_drop.head()

,year_started,year_ended,state_abbr,state,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,DataValueAlt,StratificationCategory1,Stratification1,Geolocation,LocationID,StratificationCategoryID1,StratificationID1
0,2019,2019,AR,Arkansas,BRFSS,Diabetes,Diabetes among adults,%,Crude Prevalence,13.6,13.6,Sex,Male,POINT (-92.27449074299966 34.74865012400045),5,SEX,SEXM
1,2019,2019,ID,Idaho,BRFSS,Diabetes,Diabetes among adults,%,Crude Prevalence,10.6,10.6,Sex,Male,POINT (-114.3637300419997 43.682630005000476),16,SEX,SEXM
2,2019,2019,IN,Indiana,YRBSS,Sleep,Short sleep duration among high school students,%,Crude Prevalence,NaN,NaN,Grade,Grade 12,POINT (-86.14996019399968 39.766910452000445),18,GRADE,GRD 12.00
3,2019,2019,IA,Iowa,NVSS,Asthma,"Asthma mortality among all people, underlying cause",Number,Number,54.0,54.0,Overall,Overall,POINT (-93.81649055599968 42.46940091300047),19,OVERALL,OVR
4,2019,2019,IA,Iowa,BRFSS,Asthma,Current asthma among adults,%,Crude Prevalence,10.3,10.3,Age,Age 18-44,POINT (-93.81649055599968 42.46940091300047),19,AGE,AGE1844


In [81]:
chronic_disease_df_drop.columns

Index(['year_started', 'year_ended', 'state_abbr', 'state', 'DataSource',
       'Topic', 'Question', 'DataValueUnit', 'DataValueType', 'DataValue',
       'DataValueAlt', 'StratificationCategory1', 'Stratification1',
       'Geolocation', 'LocationID', 'StratificationCategoryID1',
       'StratificationID1'],
      dtype='object')

In [82]:
chronic_disease_df_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309215 entries, 0 to 309214
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   year_started               309215 non-null  int64  
 1   year_ended                 309215 non-null  int64  
 2   state_abbr                 309215 non-null  object 
 3   state                      309215 non-null  object 
 4   DataSource                 309215 non-null  object 
 5   Topic                      309215 non-null  object 
 6   Question                   309215 non-null  object 
 7   DataValueUnit              309215 non-null  object 
 8   DataValueType              309215 non-null  object 
 9   DataValue                  209196 non-null  float64
 10  DataValueAlt               209196 non-null  float64
 11  StratificationCategory1    309215 non-null  object 
 12  Stratification1            309215 non-null  object 
 13  Geolocation                30

In [83]:
chronic_disease_df_drop['DataValue'] = chronic_disease_df_drop['DataValue'].fillna('')
chronic_disease_df_drop.head()

,year_started,year_ended,state_abbr,state,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,DataValueAlt,StratificationCategory1,Stratification1,Geolocation,LocationID,StratificationCategoryID1,StratificationID1
0,2019,2019,AR,Arkansas,BRFSS,Diabetes,Diabetes among adults,%,Crude Prevalence,13.6,13.6,Sex,Male,POINT (-92.27449074299966 34.74865012400045),5,SEX,SEXM
1,2019,2019,ID,Idaho,BRFSS,Diabetes,Diabetes among adults,%,Crude Prevalence,10.6,10.6,Sex,Male,POINT (-114.3637300419997 43.682630005000476),16,SEX,SEXM
2,2019,2019,IN,Indiana,YRBSS,Sleep,Short sleep duration among high school students,%,Crude Prevalence,,NaN,Grade,Grade 12,POINT (-86.14996019399968 39.766910452000445),18,GRADE,GRD 12.00
3,2019,2019,IA,Iowa,NVSS,Asthma,"Asthma mortality among all people, underlying cause",Number,Number,54.0,54.0,Overall,Overall,POINT (-93.81649055599968 42.46940091300047),19,OVERALL,OVR
4,2019,2019,IA,Iowa,BRFSS,Asthma,Current asthma among adults,%,Crude Prevalence,10.3,10.3,Age,Age 18-44,POINT (-93.81649055599968 42.46940091300047),19,AGE,AGE1844


In [84]:
grouped_counts = chronic_disease_df_drop.groupby(['DataSource', 'Topic'])['Topic'].count()
print(grouped_counts)

DataSource                                   Topic                                          
ACS                                          Social Determinants of Health                       3224
AEDS                                         Alcohol                                              165
ANRF                                         Tobacco                                              165
BRFSS                                        Alcohol                                            16460
                                             Arthritis                                          18645
                                             Asthma                                              4895
                                             Cancer                                              4838
                                             Cardiovascular Disease                             10117
                                             Chronic Obstructive Pulmonary Disease         

In [85]:
dataSource_df = chronic_disease_df_drop[['DataSource','state','Topic','Question', 'Geolocation', 'year_started', 'year_ended']]
dataSource_df.head()

,DataSource,state,Topic,Question,Geolocation,year_started,year_ended
0,BRFSS,Arkansas,Diabetes,Diabetes among adults,POINT (-92.27449074299966 34.74865012400045),2019,2019
1,BRFSS,Idaho,Diabetes,Diabetes among adults,POINT (-114.3637300419997 43.682630005000476),2019,2019
2,YRBSS,Indiana,Sleep,Short sleep duration among high school students,POINT (-86.14996019399968 39.766910452000445),2019,2019
3,NVSS,Iowa,Asthma,"Asthma mortality among all people, underlying cause",POINT (-93.81649055599968 42.46940091300047),2019,2019
4,BRFSS,Iowa,Asthma,Current asthma among adults,POINT (-93.81649055599968 42.46940091300047),2019,2019


In [86]:
grouped_dataSource = dataSource_df.groupby(['state', 'Topic'])['Topic'].count()
grouped_dataSource

state    Topic                                          
Alabama  Alcohol                                            467
         Arthritis                                          339
         Asthma                                             197
         Cancer                                             424
         Cardiovascular Disease                             580
                                                           ... 
Wyoming  Nutrition, Physical Activity, and Weight Status    475
         Oral Health                                        246
         Sleep                                               98
         Social Determinants of Health                      360
         Tobacco                                            295
Name: Topic, Length: 1042, dtype: int64

In [89]:
grouped_counts_df = grouped_dataSource.reset_index(name='count')
grouped_counts_df.head(40)

,state,Topic,count
0,Alabama,Alcohol,467
1,Alabama,Arthritis,339
2,Alabama,Asthma,197
3,Alabama,Cancer,424
4,Alabama,Cardiovascular Disease,580
5,Alabama,Chronic Kidney Disease,2
6,Alabama,Chronic Obstructive Pulmonary Disease,509
7,Alabama,Cognitive Health and Caregiving,180
8,Alabama,Diabetes,332
9,Alabama,Disability,92


In [103]:
sorted_grouped_counts_df = grouped_counts_df.sort_values(by=['count'], ascending=False, inplace=False)
sorted_grouped_counts_df

,state,Topic,count
402,Maryland,Cardiovascular Disease,580
516,Montana,Cardiovascular Disease,580
383,Maine,Cardiovascular Disease,580
744,Pennsylvania,Cardiovascular Disease,580
970,Washington,Cardiovascular Disease,580
...,...,...,...
100,Colorado,Chronic Kidney Disease,2
384,Maine,Chronic Kidney Disease,2
365,Louisiana,Chronic Kidney Disease,2
119,Connecticut,Chronic Kidney Disease,2


In [100]:
max_counts = sorted_grouped_counts_df.groupby('state')['count'].max()
max_counts

state
Alabama                 580
Alaska                  580
Arizona                 580
Arkansas                580
California              580
Colorado                580
Connecticut             580
Delaware                580
District of Columbia    580
Florida                 580
Georgia                 580
Guam                    460
Hawaii                  580
Idaho                   580
Illinois                580
Indiana                 580
Iowa                    580
Kansas                  580
Kentucky                580
Louisiana               580
Maine                   580
Maryland                580
Massachusetts           580
Michigan                580
Minnesota               580
Mississippi             580
Missouri                580
Montana                 580
Nebraska                580
Nevada                  580
New Hampshire           580
New Jersey              580
New Mexico              580
New York                580
North Carolina          580
North Dakota  

In [66]:
grouped_dataSource = pd.DataFrame(sorted_grouped_counts_df)
grouped_dataSource.head(50)

Topic
state   Topic                                                 
Alabama Alcohol                                            467
        Arthritis                                          339
        Asthma                                             197
        Cancer                                             424
        Cardiovascular Disease                             580
        Chronic Kidney Disease                               2
        Chronic Obstructive Pulmonary Disease              509
        Cognitive Health and Caregiving                    180
        Diabetes                                           332
        Disability                                          92
        Health Status                                      466
        Immunization                                       348
        Maternal Health                                     24
        Mental Health                                      314
        Nutrition, Physical Activity, and Weight Status    475
        Oral Health                                        246
        Sleep                                               98
        Social Determinants of Health                      360
        Tobacco                                            295
Alaska  Alcohol                                            459
        Arthritis                                          339
        Asthma                                             197
        Cancer                                             424
        Cardiovascular Disease                             580
        Chronic Kidney Disease                               2
        Chronic Obstructive Pulmonary Disease              509
        Cognitive Health and Caregiving                    180
        Diabetes                                           325
        Disability                                          92
        Health Status                                      466
        Immunization                                       348
        Maternal Health                                     17
        Mental Health                                      307
        Nutrition, Physical Activity, and Weight Status    475
        Oral Health                                        239
        Sleep                                               98
        Social Determinants of Health                      346
        Tobacco                                            288
Arizona Alcohol                                            459
        Arthritis                                          339
        Asthma                                             197
        Cancer                                             424
        Cardiovascular Disease                             580
        Chronic Kidney Disease                               2
        Chronic Obstructive Pulmonary Disease              509
        Cognitive Health and Caregiving                    180
        Diabetes                                           318
        Disability                                          92
        Health Status                                      466
        Immunization                                       348

In [40]:
dataSource_df.to_csv("csv's/dataSource_df.csv", index=False)

In [35]:
chronic_disease_df_drop.to_csv("csv's/US chronic diseases2.csv", index=False)